In [1]:
import numpy as np
import pandas as pd
import datetime
import duckdb

References:
copy warning: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
reset index: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

In [2]:
stocks = pd.read_csv("stocks_final.csv")
stocks['Year'] = pd.to_datetime(stocks['Year'], format='%Y')
stocks['Year'] = stocks['Year'].dt.year
stocks['S&P500'] = stocks['S&P500'].astype(float)

stocks.head()

,Year,S&P500
0,1928,0.44
1,1929,-0.08
2,1930,-0.25
3,1931,-0.44
4,1932,-0.09


In [3]:
incar = pd.read_csv('incarceration.csv', skiprows=5, nrows=2)
incar = incar.transpose() #switch row and column
incar.columns = incar.iloc[0] # Reset index 
incar = incar[1:]
incar.columns = ['year', 'State prisons']
incar['year'] = incar['year'].astype(int)
incar['year'] = pd.to_datetime(incar['year'], format='%Y')
incar['year'] = incar['year'].dt.year
incar['State prisons'] = incar['State prisons'].astype(float)

incar.reset_index(drop=True, inplace=True) # Reset the index
incar.head()

,year,State prisons
0,1925,85239.0
1,1926,91188.0
2,1927,101624.0
3,1928,108157.0
4,1929,107532.0


In [4]:
death = pd.read_csv('death_rates.csv')
both_sexes = death[death['Sex'] == 'Both Sexes']
death = both_sexes[['Year', 'Age-adjusted Death Rate']].copy()
death['Age-adjusted Death Rate'] = death['Age-adjusted Death Rate'].str.replace(',', '').astype(float)
death['Year'] = death['Year'].astype(int)
death['Year'] = pd.to_datetime(death['Year'], format='%Y')
death['Year'] = death['Year'].dt.year

death.head(100)

,Year,Age-adjusted Death Rate
0,1900,2518.0
1,1901,2473.1
2,1902,2301.3
3,1903,2379.0
4,1904,2502.5
...,...,...
95,1995,909.5
96,1996,893.7
97,1997,877.7
98,1998,870.1


In [5]:
birth = pd.read_csv('birth_rate.csv',skiprows=4)
birth = birth[birth['Country Name'] == 
'United States'][['Country Name', 'Indicator Name'] + [str(year) for year in range(1960, 2023)]]

birth = birth.set_index(['Country Name', 'Indicator Name']).T.reset_index() #transpose data
birth.columns = ['year', 'fertility']
birth['year'] = pd.to_datetime(birth['year'], format='%Y')
birth['fertility'] = birth['fertility'].astype(float)
birth['year'] = birth['year'].dt.year
birth.to_csv('cleaned_birth_rate_us.csv', index=False)

birth.head()

,year,fertility
0,1960,3.654
1,1961,3.620
2,1962,3.461
3,1963,3.319
4,1964,3.190


In [9]:
var = incar.merge(birth, how="left")
death = death.rename(columns={'Year': 'year'})
stocks = stocks.rename(columns={'Year': 'year'})

var = var.merge(death, how="left")
var = var.merge(stocks, how="left")
var = var.drop_duplicates(subset='year', keep='first')
var = var[35:].reset_index()
var = var.drop('index', axis=1)
print(var.tail(20))
var.to_csv("variable.csv")


    year  State prisons  fertility  Age-adjusted Death Rate  S&P500
43  2003      1256442.0     2.0475                    843.5    0.28
44  2004      1274591.0     2.0515                    813.7    0.11
45  2005      1296693.0     2.0570                    815.0    0.05
46  2006      1331065.0     2.1080                    791.8    0.16
47  2007      1353647.0     2.1200                    775.3    0.05
48  2008      1365409.0     2.0720                    774.9   -0.37
49  2009      1365688.0     2.0020                    749.6    0.26
50  2010      1362028.0     1.9310                    747.0    0.15
51  2011      1341797.0     1.8945                    741.3    0.02
52  2012      1315856.0     1.8805                    732.8    0.16
53  2013      1325305.0     1.8575                    731.9    0.32
54  2014      1316407.0     1.8625                    724.6    0.14
55  2015      1298159.0     1.8435                    733.1    0.01
56  2016      1288466.0     1.8205              